In [1]:
import pandas as pd

In [2]:
divar = pd.read_csv('divar_posts_dataset.csv' , index_col=0)
divar.head()

,archive_by_user,brand,cat1,cat2,cat3,city,created_at,desc,id,image_count,mileage,platform,price,title,type,year
0,False,NaN,for-the-home,furniture-and-home-decore,sofa-armchair,Tehran,Tuesday 07PM,کلاسیک و شیک و استثنایی\nچرم مالزی\nچوب راش\nف...,54761638662241,5,NaN,mobile,3850000,ست مبلمان و نهارخوری ٩ نفره,NaN,NaN
1,False,NaN,for-the-home,furniture-and-home-decore,antiques-and-art,Mashhad,Tuesday 07PM,"سلام,یک عدد گلدون نخل سه طبقه ی سالم دارم با پ...",42727631379133,1,NaN,mobile,30000,گلدون مصنوعی نخل,NaN,NaN
2,False,NaN,vehicles,cars,heavy,Mashhad,Tuesday 07PM,سریال 43j$NUMبدون شکستگی سه حلقه لاستیک نو یک ...,63194439667302,4,NaN,mobile,-1,لودر کاتر پیلار 950,NaN,NaN
3,False,NaN,for-the-home,furniture-and-home-decore,sofa-armchair,Tehran,Tuesday 07PM,مبل راحتی هفت نفره شامل سه نفره یک عدد دونفره ...,19133025491169,4,NaN,mobile,600000,مبل راحتی هفت نفره بامیز جلو مبلی,NaN,NaN
4,False,NaN,personal,baby-and-toys,personal-toys,Karaj,Tuesday 08PM,شارژی کنترلی سویچ حمل تا 35 کیلو صندلی برای دو...,58998998335444,2,NaN,mobile,450000,ماشین شارژی,NaN,NaN


In [5]:
divar.shape

(947635, 16)

In [6]:
divar.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 947635 entries, 0 to 947634
Data columns (total 16 columns):
archive_by_user    947635 non-null bool
brand              190892 non-null object
cat1               947635 non-null object
cat2               945877 non-null object
cat3               775301 non-null object
city               947635 non-null object
created_at         947635 non-null object
desc               947635 non-null object
id                 947635 non-null int64
image_count        947635 non-null int64
mileage            120451 non-null float64
platform           947635 non-null object
price              947635 non-null int64
title              947635 non-null object
type               98529 non-null object
year               120451 non-null object
dtypes: bool(1), float64(1), int64(3), object(11)
memory usage: 116.6+ MB


###### Start Cleaning from first column.

As we can see in the above cell 'archive_by_user' column doesn't have null values. Now we check this column for corrupted or incorrect values.

In [25]:
divar['archive_by_user'].value_counts()

True     490796
False    455081
Name: archive_by_user, dtype: int64


We see that this column doesn't have any incorrect value. In the next step, we convert bool to int for future analysis and classification or clustering.

In [3]:
divar['archive_by_user'] = divar['archive_by_user'].map({True : 1 , False : 0})
divar['archive_by_user'].value_counts()

1    491483
0    456152
Name: archive_by_user, dtype: int64

##### Brand column

We know that the brand column has many null values. If we want to delete these rows we lose very much of our data. So we should fill these null values with something. Let's look closer at the values of this column for deciding how to fill null rows.

In [74]:
divar['brand'].head(20)

0                      NaN
1                      NaN
2                      NaN
3                      NaN
4                      NaN
5                      NaN
6                      NaN
7                      NaN
8                     سایر
9                      NaN
10                     NaN
11            Nokia::نوکیا
12                     NaN
13                     NaN
14                     NaN
15                     NaN
16                     NaN
17                     NaN
18    پژو ۴۰۵::Peugeot 405
19              Apple::اپل
Name: brand, dtype: object

The not null rows in this column contains english and persian word. We want split this values and create column for each of them. Also we fill 'NAN' with 'UNKNOWN'. Because this dataset consist of multiple goods we can't fill null rows with specific item.

In [4]:
split_brand = divar['brand'].str.split('::')

In [5]:
import re
brand_en = []
brand_fa = []
for item in split_brand:
    if type(item) == list:     #for not null rows do:
        item[0] = item[0].replace("\u200c" , "")
        item[0] = item[0].replace("\u200d" , "")
        if len(item)>1:        #for rows with brand name in english and persian
            item[1] = item[1].replace("\u200c" , "")
            item[1] = item[1].replace("\u200d" , "")
            if re.match(r'[a-zA-Z0-9]' , item[0]): #check if word is english or not
                brand_en.append(item[0])
                brand_fa.append(item[1])
            else:
                brand_en.append(item[1])
                brand_fa.append(item[0])
        else:
            brand_fa.append(item[0])
            brand_en.append('other')
    else:                              #fill null values with UNKNOWN
        brand_en.append('UNKNOWN')
        brand_fa.append('UNKNOWN')

Add two columns to the dataframe and delete the brand column.

In [6]:
divar['brand_fa'] = brand_fa
divar['brand_en'] = brand_en
divar.drop('brand',axis=1,inplace=True)

##### Cat2 column

First, we check what percentage of this column is null. If it's small we can delete that rows.

In [7]:
divar['cat2'].isnull().sum()

1758

In [8]:
(divar['cat2'].isnull().sum()/divar.shape[0])*100

0.18551446495749946

In [9]:
divar.dropna(subset=['cat2'] , inplace=True)

In [10]:
divar.shape

(945877, 17)

##### Cat3 column

Because the number of rows that contain null values for the 'cat3' column is great, we don't delete them. And for this column, we fill null rows with 'UNKNOWN' value.

In [10]:
(divar['cat3'].isnull().sum()/divar.shape[0])*100

18.033634394323997

In [11]:
divar['cat3'].fillna('UNKNOWN' , inplace = True)

##### 'city' column 

This column doesn't have null values. As we can see it's values are from a few city names. Thus convert the type of it to category. 

In [52]:
divar['city'].value_counts()

Tehran        441527
Mashhad       138532
Shiraz         83970
Karaj          83736
Isfahan        61803
Ahvaz          47163
Tabriz         34014
Qom            32002
Kermanshah     23130
Name: city, dtype: int64

In [53]:
divar['city'] = divar['city'].astype('category')

##### Create_at column

The type of this column is date but it's format not suitable for analysis. We must convert it to date format.

In [12]:
divar['created_at'] = pd.to_datetime(divar['created_at'] , format = '%A %I%p')

Because the 'created_at' column doesn't have a date, we omit the date part and keep time for this column.

In [13]:
import datetime as dt
divar['created_at'] = divar['created_at'].dt.time

##### 'desc' and 'id' column 

We don't need these columns for analysis. So we delete these columns from datafram. 

In [14]:
divar.drop(columns = ['desc' , 'id'] , inplace = True)

##### 'mileage' Column

This column is used for vehicle items and for other items it is null. We need this column so we can't delete it. On the other hand, the null value of this column is too much and we can't delete these rows. So we should fill these null cells. Whereas the data type of this column is float, it's better to fill it with numeric. The minimum value of this column is '0' so we can use a negative number for filling null values.

In [15]:
divar.describe()

,archive_by_user,image_count,mileage,price
count,945877.000000,945877.000000,120451.000000,9.458770e+05
mean,0.518879,1.786155,100862.291944,3.404522e+06
std,0.499644,1.127736,98368.470916,1.372942e+07
min,0.000000,0.000000,0.000000,-1.000000e+00
25%,0.000000,1.000000,19000.000000,3.000000e+04
50%,1.000000,2.000000,75000.000000,1.500000e+05
75%,1.000000,2.000000,160000.000000,6.000000e+05
max,1.000000,16.000000,500000.000000,4.000000e+08


In [15]:
divar['mileage'].fillna(-1 , inplace = True)

##### 'platform' column

This column consists of a few distinct values. So we can convert the data type of it to category. Then convert each category to a number that can be used future for analysis.

In [16]:
divar['platform'].value_counts()

mobile    872693
web        73184
Name: platform, dtype: int64

In [18]:
divar['platform'] = divar['platform'].map({'mobile' : 0 , 'web' : 1})
divar['platform'].value_counts()

0    872693
1     73184
Name: platform, dtype: int64

##### 'price' column

This column doesn't have any null value. When we see closer to its values, we see that some rows are -1, which seems these are the same as null values. There isn't any problem seen in this column. Maybe it has outlier that we analysis them in the visualization phase.

##### "type" column 

In [23]:
divar['type'].value_counts()

women    64116
men      22980
girls     6864
boys      4569
Name: type, dtype: int64

In [25]:
(divar['type'].isnull().sum() / divar.shape[0])*100

89.58331791554292

A very large percent of the 'type' column is null. It seems that it's correlated to 'cat1'. Let's try it.

In [38]:
divar[(divar['cat1'] == 'personal') & (divar['type'].notnull())].shape

(98529, 15)

In [37]:
divar['type'].notnull().sum()

98529

In [39]:
divar[(divar['cat1'] != 'personal') & (divar['type'].notnull())].shape

(0, 15)

Our assumption is true. So we can fill null cells with a string like 'not personal'.

In [40]:
divar['type'].fillna('general' , inplace = True)

##### 'year' column

In [45]:
(divar['year'].isnull().sum() / divar.shape[0])*100

87.26568042145014

A very large percent of the 'year' column is null. It seems that it's correlated to 'mileage'. Let's try it.

In [47]:
divar[(divar['mileage'] == -1) & (divar['year'].isnull())].shape

(825426, 15)

In [48]:
divar[(divar['mileage'] != -1) & (divar['year'].isnull())].shape

(0, 15)

Our assumption is true. So we can fill null cells with a string like 'not car'.

In [49]:
divar['year'].fillna('Not Vehicle' , inplace = True)

The cleaning data is finished. Now we save our final dataset for future analysis.

In [55]:
divar.to_csv('Cleaned Divar.csv')